# Section 1: Load Setup

In [1]:
import os
import pandas as pd
import sqlite3

In [2]:
# Define file paths
full_csv = 'transformed_full.csv'
inc_csv  = 'transformed_incremental.csv'

full_db  = 'loaded_data/full_data.db'
inc_db   = 'loaded_data/incremental_data.db'

print("Paths configured:")
print(f" Full CSV       → {full_csv}")
print(f" Incremental CSV→ {inc_csv}")
print(f" Full DB        → {full_db}")
print(f" Inc DB         → {inc_db}")

Paths configured:
 Full CSV       → transformed_full.csv
 Incremental CSV→ transformed_incremental.csv
 Full DB        → loaded_data/full_data.db
 Inc DB         → loaded_data/incremental_data.db


# Section 2: Load Full Transformed Data

In [3]:
# Read the transformed full CSV, parsing order_date as datetime
df_full = pd.read_csv(full_csv, parse_dates=['order_date'], infer_datetime_format=True)
print(f"Read {len(df_full)} rows from {full_csv}")

# load full data into SQLite
conn_full = sqlite3.connect(full_db)
df_full.to_sql('full_data', conn_full, if_exists='replace', index=False)

Read 100 rows from transformed_full.csv


C:\Users\HP\AppData\Local\Temp\ipykernel_15380\3500785125.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_full = pd.read_csv(full_csv, parse_dates=['order_date'], infer_datetime_format=True)


100

# Section 3: Load Incremental Transformed Data

In [4]:
# Read the transformed incremental CSV
df_inc = pd.read_csv(inc_csv, parse_dates=['order_date'], infer_datetime_format=True)
print(f"Read {len(df_inc)} rows from {inc_csv}")

# load incremental data into SQLite
conn_inc = sqlite3.connect(inc_db)
df_inc.to_sql('incremental_data', conn_inc, if_exists='replace', index=False)
conn_inc.close()
print(f"Incremental data loaded into {inc_db} (table name: 'incremental_data')")

Read 66 rows from transformed_incremental.csv
Incremental data loaded into loaded_data/incremental_data.db (table name: 'incremental_data')


C:\Users\HP\AppData\Local\Temp\ipykernel_15380\4173419893.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_inc = pd.read_csv(inc_csv, parse_dates=['order_date'], infer_datetime_format=True)


# Section 4: Verification

In [5]:
# Verify SQLite tables
print("\nVerifying SQLite tables:")
conn = sqlite3.connect(full_db)
print(" Full row count:", pd.read_sql("SELECT COUNT(*) AS cnt FROM full_data;", conn))
display(pd.read_sql("SELECT * FROM full_data LIMIT 5;", conn))
conn.close()

conn = sqlite3.connect(inc_db)
print(" Incremental row count:", pd.read_sql("SELECT COUNT(*) AS cnt FROM incremental_data;", conn))
display(pd.read_sql("SELECT * FROM incremental_data LIMIT 5;", conn))
conn.close()


Verifying SQLite tables:
 Full row count:    cnt
0  100


,order_id,product,order_date,quantity,total_price,order_year,order_month,order_day,avg_price_per_item,price_category
0,1,Tablet,2025-06-26 11:18:00,1,583.28,2025,6,26,583.28,Medium
1,2,Camera,2025-06-13 11:27:00,4,2299.20,2025,6,13,574.80,Medium
2,3,Headphones,2025-06-23 22:30:00,3,3682.14,2025,6,23,1227.38,High
3,4,Headphones,2025-06-10 14:30:00,1,687.12,2025,6,10,687.12,Medium
4,5,Smartphone,2025-06-26 02:40:00,1,990.39,2025,6,26,990.39,Medium


 Incremental row count:    cnt
0   66


,order_id,product,order_date,quantity,total_price,order_year,order_month,order_day,avg_price_per_item,price_category
0,1,Tablet,2025-06-26 11:18:00,1,583.28,2025,6,26,583.2800,Medium
1,2,Camera,2025-06-13 11:27:00,4,2299.20,2025,6,13,574.8000,Medium
2,3,Headphones,2025-06-23 22:30:00,3,3682.14,2025,6,23,1227.3800,High
3,5,Smartphone,2025-06-26 02:40:00,1,990.39,2025,6,26,990.3900,Medium
4,6,Camera,2025-07-05 00:43:00,4,3613.93,2025,7,5,903.4825,Medium
